# Coverage analysis

In [ ]:
import ephemerista
from ephemerista.analysis.coverage import Coverage
from ephemerista.assets import Asset, Spacecraft
from ephemerista.propagators.sgp4 import SGP4
from ephemerista.scenarios import Scenario
from ephemerista.time import TimeDelta

In [ ]:
ephemerista.init(eop_path="../tests/resources/finals2000A.all.csv", spk_path="../tests/resources/de440s.bsp")

In [ ]:
tle = """SENTINEL-6
1 46984U 20086A   24319.21552651 -.00000061  00000+0  71254-6 0  9995
2 46984  66.0411 259.6585 0007844 270.2444  89.7673 12.80930600186045
"""

propagator = SGP4(tle=tle)
sc = Asset(model=Spacecraft(propagator=propagator), name="PHASMA")

In [ ]:
start_time = propagator.time
end_time_one_orbit = start_time + TimeDelta.from_minutes(112)
end_time_longer = start_time + TimeDelta.from_hours(24 * 5)

### Option 1: load multi polygon (PolygonFeatureCollection) from geojson file

The `load_geojson_multipolygon` method takes an additional argument `min_elevation_deg` to define at which minimum elevation from the ground location the visibility must be computed. Here we use a value of 0° to model a ground station. 

In [ ]:
from ephemerista.analysis.coverage import load_geojson_multipolygon

feature_list = load_geojson_multipolygon("simple_polygons.geojson", min_elevation_deg=0.0)

In [ ]:
scenario_a_few_polygons = Scenario(
    assets=[sc],
    name="Coverage analysis",
    start_time=start_time,
    end_time=end_time_longer,
    areas_of_interest=feature_list,
    auto_discretize=False,
)

In [ ]:
cov_a_few_polygons = Coverage(scenario=scenario_a_few_polygons)
results_a_few_polygons = cov_a_few_polygons.analyze()

In [ ]:
display(results_a_few_polygons.to_geodataframe())

#### Matplotlib: coverage percentage

In [ ]:
import matplotlib.pyplot as plt

ax = results_a_few_polygons.plot_mpl()
plt.show()

#### Plotly: coverage percentage

In [ ]:
fig = results_a_few_polygons.plot_plotly()
fig.show()

#### Plotly: maximum time gap

In [ ]:
fig = results_a_few_polygons.plot_plotly(data_to_plot="max_time_gaps")
fig.show()

### Option 2: using h3 to polygonize a region

The `polygonize_aoi` method takes an additional argument `min_elevation_deg` to define at which minimum elevation from the ground locations the visibility must be computed.

In this example we use 70° minimum elevation to model an optical satellite which can depoint a bit off-nadir but not too much in order not to loose too much image quality.

In [ ]:
import geojson_pydantic

with open("single_aoi.geojson") as f:
    aoi = geojson_pydantic.FeatureCollection.model_validate_json(f.read())

In [ ]:
from ephemerista.scenarios import polygonize_aoi, polygonize_aoi_rectangles

feature_list = polygonize_aoi(
    aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"], res=1, min_elevation_deg=70.0
)

### One orbit

In [ ]:
scenario_one_orbit = Scenario(
    assets=[sc],
    name="Coverage analysis",
    start_time=start_time,
    end_time=end_time_one_orbit,
    areas_of_interest=feature_list,
    auto_discretize=False,
)

In [ ]:
cov_one_orbit = Coverage(scenario=scenario_one_orbit)
results_one_orbit = cov_one_orbit.analyze()

In [ ]:
display(results_one_orbit.to_geodataframe())

#### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

ax = results_one_orbit.plot_mpl(legend=True, cmap="viridis")
plt.show()

#### Plotly: coverage percentage

In [ ]:
fig = results_one_orbit.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()

#### Plotly: worst time gaps in days

The cells without any coverage during the simulation are not visible. Cells with only one visibility event during the simulation receive a time gap equal to the simulation duration, because it is not possible to compute the real time gap.

In [ ]:
fig = results_one_orbit.plot_plotly(data_to_plot="max_time_gaps", color_continuous_scale="Jet", opacity=0.4)
fig.show()

### 5 days

In [ ]:
scenario_longer = Scenario(
    assets=[sc],
    name="Coverage analysis",
    start_time=start_time,
    end_time=end_time_longer,
    areas_of_interest=feature_list,
    auto_discretize=False,
)

In [ ]:
cov_longer = Coverage(scenario=scenario_longer)
results_longer = cov_longer.analyze()

#### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

ax = results_longer.plot_mpl(legend=True, cmap="viridis")
plt.show()

#### Plotly: coverage percentage

In [ ]:
fig = results_longer.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()

#### Plotly: worst time gaps in days

The cells without any coverage during the simulation are not visible.

In [ ]:
fig = results_longer.plot_plotly(data_to_plot="max_time_gaps", color_continuous_scale="jet", opacity=0.4)
fig.show()

### Option 3: using rectangles to polygonize a region

The `polygonize_aoi_rectangles` method takes an additional argument `min_elevation_deg` to define at which minimum elevation from the ground locations the visibility must be computed.

In this example we use 70° minimum elevation to model an optical satellite which can depoint a bit off-nadir but not too much in order not to loose too much image quality.

In [ ]:
import geojson_pydantic

with open("single_aoi.geojson") as f:
    aoi = geojson_pydantic.FeatureCollection.model_validate_json(f.read())

In [ ]:
feature_list = polygonize_aoi_rectangles(
    aoi_geom_dict=aoi.__geo_interface__["features"][0]["geometry"], vertex_degrees=5, min_elevation_deg=70.0
)

### One orbit

In [ ]:
scenario_one_orbit = Scenario(
    assets=[sc],
    name="Coverage analysis",
    start_time=start_time,
    end_time=end_time_one_orbit,
    areas_of_interest=feature_list,
    auto_discretize=False,
)

In [ ]:
cov_one_orbit = Coverage(scenario=scenario_one_orbit)
results_one_orbit = cov_one_orbit.analyze()

In [ ]:
display(results_one_orbit.to_geodataframe())

#### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

ax = results_one_orbit.plot_mpl(legend=True, cmap="viridis")
plt.show()

#### Plotly: coverage percentage

In [ ]:
fig = results_one_orbit.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()

#### Plotly: worst time gaps in days

The cells without any coverage during the simulation are not visible. Cells with only one visibility event during the simulation receive a time gap equal to the simulation duration, because it is not possible to compute the real time gap.

In [ ]:
fig = results_one_orbit.plot_plotly(data_to_plot="max_time_gaps", color_continuous_scale="Jet", opacity=0.4)
fig.show()

### 5 days

In [ ]:
scenario_longer = Scenario(
    assets=[sc],
    name="Coverage analysis",
    start_time=start_time,
    end_time=end_time_longer,
    areas_of_interest=feature_list,
    auto_discretize=False,
)

In [ ]:
cov_longer = Coverage(scenario=scenario_longer)
results_longer = cov_longer.analyze()

In [ ]:
display(results_longer.to_geodataframe())

#### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

ax = results_longer.plot_mpl(legend=True, cmap="viridis")
plt.show()

#### Plotly: coverage percentage

In [ ]:
fig = results_longer.plot_plotly(color_continuous_scale="Jet", opacity=0.5)
fig.show()

#### Plotly: worst time gaps in days

The cells without any coverage during the simulation are not visible.

In [ ]:
fig = results_longer.plot_plotly(data_to_plot="max_time_gaps", color_continuous_scale="jet", opacity=0.4)
fig.show()